In [22]:
import pandas as pd

import plotly.express as pe

#model

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split

from hyperopt import hp,tpe,fmin,Trials,STATUS_OK,space_eval
from hyperopt.early_stop import no_progress_loss

#metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

### I want to market high risk high rewards mutual funds investment plan
### The plan requires heavy investment every year
### the plan is aimed at individuals who have more free cash available and can invest for a lot of years

In [23]:
path = r"C:\Users\harsh\Desktop\NPCI-Python-ML\datasets\Loan_Status_Classification.csv"

df = pd.read_csv(path)

df

Gender  Married  Dependents  Education  Self_Employed  ApplicantIncome  \
0         0        1           1          0              0             6608   
1         0        1           2          0              0             4226   
2         1        1           0          1              0             3167   
3         0        0           0          1              1             6950   
4         0        1           0          1              0             3993   
..      ...      ...         ...        ...            ...              ...   
293       1        0           0          1              0             3846   
294       0        0           0          1              0             2435   
295       0        0           2          1              0             4923   
296       0        1           3          0              0             2071   
297       1        0           0          1              0             2378   

     CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
0                    0         137               180               1   
1                 1040         110               360               1   
2                 2283         154               360               1   
3                    0         175               180               1   
4                 3274         207               360               1   
..                 ...         ...               ...             ...   
293                  0         111               360               1   
294                  0          75               360               1   
295                  0         166               360               0   
296                754          94               480               1   
297                  0          46               360               1   

     Property_Area  Loan_Status  
0                1            1  
1                1            1  
2                2            1  
3                2            1  
4                2            1  
..             ...          ...  
293              2            1  
294              1            0  
295              2            1  
296              2            1  
297              3            0  

[298 rows x 12 columns]

### step 2: Data exploration

In [24]:
df.shape

(298, 12)

In [25]:
df.index

RangeIndex(start=0, stop=298, step=1)

In [26]:
df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Gender             298 non-null    int64
 1   Married            298 non-null    int64
 2   Dependents         298 non-null    int64
 3   Education          298 non-null    int64
 4   Self_Employed      298 non-null    int64
 5   ApplicantIncome    298 non-null    int64
 6   CoapplicantIncome  298 non-null    int64
 7   LoanAmount         298 non-null    int64
 8   Loan_Amount_Term   298 non-null    int64
 9   Credit_History     298 non-null    int64
 10  Property_Area      298 non-null    int64
 11  Loan_Status        298 non-null    int64
dtypes: int64(12)
memory usage: 28.1 KB


In [28]:
df.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [29]:
df.nunique()

Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      257
CoapplicantIncome    150
LoanAmount           145
Loan_Amount_Term       9
Credit_History         2
Property_Area          3
Loan_Status            2
dtype: int64

In [30]:
df["Loan_Status"].value_counts()

1    150
0    148
Name: Loan_Status, dtype: int64

In [31]:
df.columns = [   col.replace("_", "") for col in df.columns    ]
print(df.columns)

mapping = {"CoapplicantIncome" : "CoApplicantIncome"}


df.rename(  columns = mapping, inplace=True        )

df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'SelfEmployed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'LoanAmountTerm',
       'CreditHistory', 'PropertyArea', 'LoanStatus'],
      dtype='object')


Index(['Gender', 'Married', 'Dependents', 'Education', 'SelfEmployed',
       'ApplicantIncome', 'CoApplicantIncome', 'LoanAmount', 'LoanAmountTerm',
       'CreditHistory', 'PropertyArea', 'LoanStatus'],
      dtype='object')

#### step 3: feature & target selection

In [32]:
features = ["ApplicantIncome", "CoApplicantIncome", "LoanAmount"]
target = "LoanStatus"

### methods for splitting data into (training, testing/validation)

train_test_split : stratified ratio-based direct splitting. gives only 1 set each for training & testing 

Kfold : split the data K times. Gives K set each for training & testing
    -
    -
KFOLDCV : 
    -
    -
STRATIFIED K FOLD CV:
    -
    -

In [33]:
x_train, x_test, y_train, y_test = train_test_split(   
    df[features], 
    df[target], 
    test_size=0.2 , 
    random_state=10, 
    stratify=df[target]  
) #

## step 5 training the model

In [34]:
model = KNeighborsClassifier()

### KNN is a lazy algorithm!

In [35]:
model.fit(x_train, y_train)

KNeighborsClassifier()

### BONUS STEP : check accuracy of model during training

### step 6 : evaluate the model

In [36]:
ans = model.predict(     x_test      )

ans_df = pd.DataFrame(ans,   columns = ["PredictedStatus"])

x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

#concatenating features from testing points, actual answers in testing points & predictions 
#made by the model on the testing points
result_df = pd.concat([ x_test, y_test, ans_df   ],axis=1)

result_df

ApplicantIncome  CoApplicantIncome  LoanAmount  LoanStatus  \
0              2330               4486         100           1   
1              3316               3500          88           1   
2              3167               2283         154           1   
3              8333                  0         280           1   
4              4009               1717         116           1   
5              9538                  0         187           1   
6              1000               3022         110           0   
7              2130               6666          70           0   
8              3522                  0          81           0   
9              5726               4595         258           0   
10             6950                  0         175           1   
11             2708               1167          97           1   
12             3400               2500         123           0   
13             4616                  0         134           0   
14             3993               3274         207           1   
15             3357               2859         144           1   
16             4000               7750         290           0   
17             3125               2583         170           0   
18             4895                  0         102           1   
19             7787                  0         240           1   
20             4583               1508         128           0   
21             6033                  0         160           0   
22             4583                  0          84           0   
23             4692                  0         106           0   
24             6277                  0         118           0   
25              150               1800         135           0   
26             4300                  0         136           0   
27             1500               1800         103           0   
28             2333               1451         102           0   
29             2583               2358         120           1   
30             4191                  0         120           1   
31            19730               5266         570           0   
32             3358                  0          80           0   
33            10000                  0         155           0   
34             3988                  0          50           1   
35             7933                  0         275           0   
36             8799                  0         258           0   
37             2178                  0          66           0   
38            14683               2100         304           0   
39             7200                  0         120           1   
40             5649                  0          44           1   
41             4095               3447         151           1   
42             4000               2275         144           1   
43             3000               1666         100           0   
44             4288               3263         133           1   
45             1299               1086          17           1   
46             8080               2250         180           1   
47             2014               1929          74           1   
48             4408                  0         120           1   
49             2600               2500          90           1   
50             3500               1667         114           1   
51             6250               5654         188           1   
52             3500                  0          81           1   
53             2965               5701         155           1   
54             5568               2142         175           0   
55             2653               1500         113           0   
56             2378                  0           9           0   
57             1928               1644         100           1   
58             1800               2934          93           0   
59             2600               1717      

In [37]:
print ( confusion_matrix(    y_test, ans_df    ) )

[[19 11]
 [15 15]]


In [38]:
print ( classification_report(    y_test, ans_df    ) )

              precision    recall  f1-score   support

           0       0.56      0.63      0.59        30
           1       0.58      0.50      0.54        30

    accuracy                           0.57        60
   macro avg       0.57      0.57      0.56        60
weighted avg       0.57      0.57      0.56        60



In [39]:
df["LoanStatus"] = df["LoanStatus"].astype("object")

pe.scatter(
    x="ApplicantIncome",
    y="LoanAmount",
    color="LoanStatus",
    size = "CoApplicantIncome",
    data_frame=df
)

In [40]:
#keys of this dictionary are names of parameters
space ={
    "n_neighbors" : hp.choice("neighbour count", [x for x in range(1,25,1)]),
    'weights' : hp.choice(  'depth parameter',["distance", "uniform"]  ), #choose some number between 1 to 8   
}

In [41]:
scores=[]
def objective(space):
    model = KNeighborsClassifier(**space)

    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    accuracy = accuracy_score(y_test,pred) #-57 -70: hyperopt selects -70


    scores.append(accuracy)
    return {'loss': -accuracy,'status':STATUS_OK}

In [42]:
trials = Trials() #create a database (in-memory)

      #fmin is the function to be used for minimum optimization
best = fmin(fn=objective, #function
            space=space, #search space parameters
            algo=tpe.suggest, #next combination has to be picked intelligently(bayesian optimization)
            max_evals=420,
            early_stop_fn=no_progress_loss(  iteration_stop_count=50, percent_increase=0.0001 ), 
            trials=trials)

print (space_eval(space, best))

 13%|█▎        | 55/420 [00:00<00:04, 89.57trial/s, best loss: -0.6833333333333333]
{'n_neighbors': 3, 'weights': 'distance'}


P(h|D)

P(whether sunny |  sports is no ) 


=  P(sports = NO given whether is sunny) * P (whether = sunny)
  ---------------------------------------------------------------
     P(sports is not to be played)

P (h|D)


P (sports = no given that whether = sunny)
= P(whether = sunny given that sports = NO) * P(sports = No)
-----------------------------------------------------------
          p(whether = sunny)